# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install -q crewai==0.108.0 crewai_tools==0.38.1 langchain_community==0.3.20 langchain_ollama  ollama==0.4.7 python-dotenv
!pip install --upgrade langchain_openai
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew, LLM

2025-03-25 20:50:24,815 - 6246068224 - __init__.py-__init__:362 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/skasmani/miniforge3/lib/python3.12/http/client.py", line 1428, in getresponse
    response.begin()
  File "/Users/skasmani/mi

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `mistral-small:24b`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
# import os
# from utils import get_openai_api_key, pretty_print_result
# from dotenv import load_dotenv

# load_dotenv('creds.env')  # Adjust path if needed


# from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = "testapikey"

# ollama_llm = ChatOpenAI(
# model="ollama/mistral-small:24b",
# base_url="http://localhost:11434"
# )


In [4]:
import os
from utils import get_openai_api_key, pretty_print_result
from dotenv import load_dotenv

load_dotenv('creds.env')  # Adjust path if needed


ollama_llm = LLM(
  	model="ollama/phi4:latest",
  	api_base="http://localhost:11434"
  )


## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/skasmani/Downloads/personal/github/C

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    llm=ollama_llm,
    allow_delegation=True,
    max_iter=5,
    memory=True, 
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    llm=ollama_llm,
    allow_delegation=True,
    max_iter=5,
    memory=True, 
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    max_iter=5,
    memory=True, 
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    llm=ollama_llm,
    allow_delegation=True,
    memory=True, 
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ollama_llm,
    process=Process.hierarchical,
    embedder={
        "provider": "ollama",
        "config": {
            "model": "mxbai-embed-large"
        }
    },
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 57921e71-c578-4745-bb5f-d954fcfb4ad6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4841ede0-a52a-4c0a-9ccf-e42b840cfc01
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4841ede0-a52a-4c0a-9ccf-e42b840cfc01
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


🚀 Crew: crew
└── 📋 Task: 4841ede0-a52a-4c0a-9ccf-e42b840cfc01
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        └── 🧠 Thinking...

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 57921e71-c578-4745-bb5f-d954fcfb4ad6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open Interest Highest Implied Volatility Stock Comparison Advanced C

Final Answer: 
Apple Inc. (AAPL) is a leading technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products, including the iPhone, Mac, iPad, and various accessories like AirPods and Apple Watch. Additionally, Apple provides services such as AppleCare support, cloud services, the App Store for downloading applications and digital content, and subscription services like Apple Arcade, Apple Fitness+, Apple Music, and Apple TV+.

As of the most recent data, Apple's stock price is $191.06, with a market cap of $2.93 trillion. The company has shown consistent growth over the years, with a 1-year return of 9.49% and a 5-year return of 344.30%. Apple's financial highlights include a profit margin of 26.31%, return on assets of 22.07%, and revenue of $381.62 billion. The company also has a strong balance sheet with total cash of $67.15 billion and levered free cash flow of $84.73

Final Answer: I have gathered insights on AAPL from the Investopedia website. The next step is to analyze the information provided and develop potential trading strategies for day trading based on the user's risk tolerance.

> Finished chain.
 

I have gathered insights on AAPL from the Investopedia website. The next step is to analyze the information provided and develop potential trading strategies for day trading based on the user's risk tolerance.

Final Answer: 
After analyzing the insights gathered on AAPL from various sources, including the Investopedia website, I have developed the following potential trading strategies for AAPL that align with the user's medium risk tolerance and preference for Day Trading:

1. Momentum Trading Strategy: This strategy involves buying AAPL shares when the stock price shows upward momentum and selling when it shows signs of slowing down. It is suitable for short-term traders looking to capitalize on short price movements.

2. Breakout Trading St

Thought: I have gathered information on potential trading strategies for AAPL from the internet search results.

Final Answer: The search results provided insights into various trading strategies for AAPL, such as buying on weakness to certain value levels, using options strategies like the calendar spread, and considering the company's fundamentals and technical characteristics. It is essential to analyze these strategies further to determine the best alignment with the current market conditions, taking into account factors like implied volatility, earnings, and support/resistance levels.

> Finished chain.
 

The search results provided insights into various trading strategies for AAPL, such as buying on weakness to certain value levels, using options strategies like the calendar spread, and considering the company's fundamentals and technical characteristics. It is essential to analyze these strategies further to determine the best alignment with the current market conditions, takin

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
# Extract the Markdown string from the CrewOutput object
markdown_content = result.raw

# Display the Markdown content
display(Markdown(markdown_content))
# Markdown(result)

Apple’s future growth potential and stock value are promising, driven by its strong historical performance, brand loyalty, strategic investments in R&D, and focus on emerging market trends like digital services and wearables. Despite facing challenges from competition and regulatory scrutiny, Apple's robust financial health, integrated ecosystem, and commitment to innovation position it well for sustained success. As long as the company continues to adapt to consumer needs and manages external pressures effectively, its stock value is expected to appreciate in the long run.